In [1]:
import pandas as pd
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler

from time_series_experiments import utils
from time_series_experiments import baseline
from time_series_experiments import transformer

In [2]:
seed = 0xC0FFEE
fdw = 24
fw = 12

air_df = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv')

In [3]:
y = air_df['Passengers'].values

x_train_idx, y_train_idx, x_test_idx, y_test_idx = utils.train_test_split_index(
    y.shape[0], fdw, fw, test_size=0.1, random_seed=seed
)

x_train = np.expand_dims(y[x_train_idx], axis=-1)
y_train = y[y_train_idx]
x_test = np.expand_dims(y[x_test_idx], axis=-1)
y_test = y[y_test_idx]

In [4]:
baseline_model = baseline.LatestNaiveBaseline(fw=fw)
y_pred_baseline = baseline_model.predict(x_test)
utils.rmse(y_test, np.squeeze(y_pred_baseline))

96.54480824984842

In [25]:
scaler = MinMaxScaler(feature_range=(0.01, 0.99))

xtr, ytr = x_train, y_train 
# xtr, ytr = np.log(xtr), np.log(ytr)
scaler.fit(np.reshape(xtr, (-1, 1)))
original_x_shape = xtr.shape
original_y_shape = ytr.shape
xtr = scaler.transform(np.reshape(xtr, (-1, 1)))
ytr = scaler.transform(np.reshape(ytr, (-1, 1)))
xtr = np.reshape(xtr, original_x_shape)
ytr = np.reshape(ytr, original_y_shape)

transformer_model = transformer.Transformer(
    num_layers=1,
    attention_dim=4,
    num_heads=4,
    linear_kernel_initializer=utils.get_initializer("glorot_uniform", seed),
    attention_kernel_initializer=utils.get_initializer("glorot_uniform", seed),
    pwffn_kernel_initializer=utils.get_initializer("glorot_uniform", seed),
    output_kernel_initializer=utils.get_initializer("glorot_uniform", seed),
    epochs=100,
    batch_size=32,
    optimizer=keras.optimizers.Adam(0.01),
    loss=keras.losses.MeanAbsoluteError(),
)
transformer_model.fit(xtr, ytr, verbose=1)

Train on 99 samples
Epoch 1/100
99/99 [==============================] - 2s 21ms/sample - loss: 1.6511
Epoch 2/100
99/99 [==============================] - 0s 235us/sample - loss: 0.7011
Epoch 3/100
99/99 [==============================] - 0s 282us/sample - loss: 0.4051
Epoch 4/100
99/99 [==============================] - 0s 274us/sample - loss: 0.2323
Epoch 5/100
99/99 [==============================] - 0s 273us/sample - loss: 0.2182
Epoch 6/100
99/99 [==============================] - 0s 239us/sample - loss: 0.2429
Epoch 7/100
99/99 [==============================] - 0s 248us/sample - loss: 0.1943
Epoch 8/100
99/99 [==============================] - 0s 247us/sample - loss: 0.1912
Epoch 9/100
99/99 [==============================] - 0s 245us/sample - loss: 0.1770
Epoch 10/100
99/99 [==============================] - 0s 236us/sample - loss: 0.1897
Epoch 11/100
99/99 [==============================] - 0s 241us/sample - loss: 0.1790
Epoch 12/100
99/99 [==============================] - 0

In [26]:
xt = x_test
# xt = np.log(xt)
original_x_shape = xt.shape
xt = scaler.transform(np.reshape(xt, (-1, 1)))
xt = np.reshape(xt, original_x_shape)

y_pred, _ = transformer_model.predict(xt)
y_pred = scaler.inverse_transform(y_pred)
# y_pred = np.exp(y_pred)

'RMSE', utils.rmse(y_test, y_pred), 'MASE', utils.mase(y_test, y_pred, np.squeeze(y_pred_baseline))

('RMSE', 64.66266019105187, 'MASE', 0.7181447631203906)